## Setup
This notebook requires that you have a copy of the AACT data set up in a MySQL database. To make this as easy as possible I've created some simple scripts to perform all necessary tasks to download, extract and transform the data into a MySql database. The code and directions can be found at: [https://github.com/dansbits/aact_analysis](https://github.com/dansbits/aact_analysis)

This analysis was completed using the March 2016 release of AACT.

In [2]:
require 'dbi'
require 'daru'
require 'nyaplot'
require 'yaml'

# change this to point to your aact_analysis directory
database = YAML.load_file("/home/dan/workspace/aact_analysis/secrets.yml")['database']; nil

db = DBI.connect("dbi:Mysql:#{database['name']}:#{database['host']}", database['user'], database['password']); nil

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    win

## ClinicalTrials.gov - FDAAA Section 801 Compliance

This analysis takes a look at the Food, Drug and Cosmetic Adminisration Act and compliance with some of the regulations it puts in place. Specifically I'm interested in measuring adherence with posting of results to the results data bank.

It first became possible to post results to ClinicalTrials.gov in September 2008. I'd first like to look into how often trials post results without taking into account whether it's required or not.

In [4]:
studies = Daru::DataFrame.from_sql(
  db, 
  <<-SQL
    select
      cs.nct_id,
      start_date,
      primary_completion_date,
      firstreceived_results_date,
      is_section_801,
      case 
        when firstreceived_results_date is null or firstreceived_results_date = '' then 'No results submitted'
        when firstreceived_results_date is not null then 'Results submitted' 
      end as results_status
    from clinical_study cs
    where
      -- include studies which started after 9/1/2008 or completed after completed after 9/1/2008
      (
        start_date is not null 
        and start_date > '2008-09-01'
      )
      and (
        primary_completion_date is not null 
        and primary_completion_date < '2015-01-01'
      )
  SQL
)

studies.size

64753

In [6]:
overall_results_rates = studies.group_by(:results_status).count[1..1]
overall_results_rates.vectors = Daru::Index.new([:total_study_count])
overall_results_rates[:results_status] = overall_results_rates.index.to_a

overall_results_rates[:percent_of_total] = overall_results_rates.map(:row) do |r| 
  ((r[:total_study_count].to_f / studies.size.to_f) * 100.0).round(4)
end

overall_results_rates.plot type: :bar, x: :results_status, y: :total_study_count do |plot, diagram|
  plot.x_label "Results Status"
  plot.y_label "Total Studies"
end

#<Nyaplot::Frame:0x0000000bb0b570 @properties={:panes=>[#<Nyaplot::Plot:0x0000000bb01390 @properties={:diagrams=>[#<Nyaplot::Diagram:0x0000000bb0bb10 @properties={:type=>:bar, :options=>{:x=>:results_status, :y=>:total_study_count}, :data=>"bc1b11a2-5f22-4023-8585-8ee9d3511030"}, @xrange=["No results submitted", "Results submitted"], @yrange=[0, 54782]>], :options=>{:x_label=>"Results Status", :y_label=>"Total Studies", :width=>700, :xrange=>["No results submitted", "Results submitted"], :yrange=>[0, 54782]}}>], :data=>{"bc1b11a2-5f22-4023-8585-8ee9d3511030"=>#<Nyaplot::DataFrame:0x0000000bb0bf48 @name="bc1b11a2-5f22-4023-8585-8ee9d3511030", @rows=[{:total_study_count=>54782, :results_status=>"No results submitted", :percent_of_total=>84.6015}, {:total_study_count=>9971, :results_status=>"Results submitted", :percent_of_total=>15.3985}]>}, :extension=>[]}>

Only about 4.25% of studies have any results submitted. Let's see how this has been trending over time.

In [7]:
studies[:completion_year] = studies.map(:row) { |r| r[:primary_completion_date].year }

results_rates_by_year = studies.group_by(:completion_year, :results_status).count[1..1]
results_rates_by_year.vectors = Daru::Index.new([:total_study_count])
results_rates_by_year[:completion_year] = results_rates_by_year.index.to_a.map { |i| i[0] }
results_rates_by_year[:results_status] = results_rates_by_year.index.to_a.map { |i| i[1] }

results_rates_by_year[:yearly_percent] = results_rates_by_year.map(:row) do |year_row|
  ((year_row[:total_study_count].to_f / studies.filter(:row) { |study_row| year_row[:completion_year] == study_row[:completion_year] }.size.to_f) * 100.0).round(4)
end

plot_data = results_rates_by_year.filter(:row) { |r| r[:results_status] == 'Results submitted' }
plot_data[:completion_year] = plot_data[:completion_year].map { |y| y.to_s }

plot_data.plot type: :bar, x: :completion_year, y: :yearly_percent do |plot, diagram|
  plot.x_label "Primary Completion Year"
  plot.y_label "% of Studies with Posted Results"
end

#<Nyaplot::Frame:0x0000000a880c48 @properties={:panes=>[#<Nyaplot::Plot:0x0000000a896598 @properties={:diagrams=>[#<Nyaplot::Diagram:0x0000000a8811e8 @properties={:type=>:bar, :options=>{:x=>:completion_year, :y=>:yearly_percent}, :data=>"2f4cece0-a724-47ac-a187-d6258c34b43f"}, @xrange=["2008", "2009", "2010", "2011", "2012", "2013", "2014"], @yrange=[0, 19.8333]>], :options=>{:x_label=>"Primary Completion Year", :y_label=>"% of Studies with Posted Results", :width=>700, :xrange=>["2008", "2009", "2010", "2011", "2012", "2013", "2014"], :yrange=>[0, 19.8333]}}>], :data=>{"2f4cece0-a724-47ac-a187-d6258c34b43f"=>#<Nyaplot::DataFrame:0x0000000a881990 @name="2f4cece0-a724-47ac-a187-d6258c34b43f", @rows=[{:total_study_count=>23, :completion_year=>"2008", :results_status=>"Results submitted", :yearly_percent=>17.8295}, {:total_study_count=>616, :completion_year=>"2009", :results_status=>"Results submitted", :yearly_percent=>19.6303}, {:total_study_count=>1428, :completion_year=>"2010", :results_status=>"Results submitted", :yearly_percent=>19.8333}, {:total_study_count=>1852, :completion_year=>"2011", :results_status=>"Results submitted", :yearly_percent=>17.8317}, {:total_study_count=>2146, :completion_year=>"2012", :results_status=>"Results submitted", :yearly_percent=>16.8512}, {:total_study_count=>2138, :completion_year=>"2013", :results_status=>"Results submitted", :yearly_percent=>14.5225}, {:total_study_count=>1768, :completion_year=>"2014", :results_status=>"Results submitted", :yearly_percent=>10.7523}]>}, :extension=>[]}>

This shows that a smaller percentage of studies have been posting results since 2008. It's important to note though that posting results is not required for all of these studies and those plot doesn't consider what percentage of studies are required to share results. While many would argue there is an ethical obligation for all studies to post results, there is not a legal mandate for all studies.

Its also worth noting that there have been increasingly
more studies registered and completed on ClinicalTrials.gov each year since 2008.

In [8]:
yearly_study_completions = studies.group_by(:completion_year).count[1..1]
yearly_study_completions.vectors = Daru::Index.new([:total_study_count])
yearly_study_completions[:completion_year] = yearly_study_completions.index.to_a.map { |y| y.to_s }
yearly_study_completions

yearly_study_completions.plot type: :bar, x: :completion_year, y: :total_study_count do |plot, diagram|
  plot.x_label 'Year'
  plot.y_label 'Total studies completed'
end

#<Nyaplot::Frame:0x000000083d0d08 @properties={:panes=>[#<Nyaplot::Plot:0x000000083e0410 @properties={:diagrams=>[#<Nyaplot::Diagram:0x000000083d17a8 @properties={:type=>:bar, :options=>{:x=>:completion_year, :y=>:total_study_count}, :data=>"d7cb0ee9-1776-4bb3-ad16-933b0f850890"}, @xrange=["2008", "2009", "2010", "2011", "2012", "2013", "2014"], @yrange=[0, 16443]>], :options=>{:x_label=>"Year", :y_label=>"Total studies completed", :width=>700, :xrange=>["2008", "2009", "2010", "2011", "2012", "2013", "2014"], :yrange=>[0, 16443]}}>], :data=>{"d7cb0ee9-1776-4bb3-ad16-933b0f850890"=>#<Nyaplot::DataFrame:0x000000083d2450 @name="d7cb0ee9-1776-4bb3-ad16-933b0f850890", @rows=[{:total_study_count=>129, :completion_year=>"2008"}, {:total_study_count=>3138, :completion_year=>"2009"}, {:total_study_count=>7200, :completion_year=>"2010"}, {:total_study_count=>10386, :completion_year=>"2011"}, {:total_study_count=>12735, :completion_year=>"2012"}, {:total_study_count=>14722, :completion_year=>"2013"}, {:total_study_count=>16443, :completion_year=>"2014"}]>}, :extension=>[]}>

# FDAAA Section 801 Compliance

The above analysis shows that results are submitted by very few studies overall. However, it's not required for all studies. For many studies included above there is no legal obligation to post results. Let's take a look at those which ARE required to submit results and see how compliant they are. In September 2009 it became a requirement that applicable studies under FDAAA section 801 submit adverse events and final results to the registry.

Studies are not required to submit results until 1 year after the primary completion date so I will exclude studies completed studies which have not passed this grace period yet.

In [15]:
regulated_studies = Daru::DataFrame.from_sql(
  db, 
  <<-SQL
    select
  cs.nct_id,
  start_date,
  primary_completion_date,
  firstreceived_results_date,
  overall_status,
  DATE_ADD(primary_completion_date, INTERVAL 1 YEAR) AS results_due_date,
  is_section_801,
  case 
    when firstreceived_results_date is null or firstreceived_results_date = '' then 'No results submitted'
    when firstreceived_results_date is not null and firstreceived_results_date <= DATE_ADD(primary_completion_date, INTERVAL 1 YEAR)  then 'Results submitted on time' 
    when firstreceived_results_date is not null and firstreceived_results_date > DATE_ADD(primary_completion_date, INTERVAL 1 YEAR)  then 'Results submitted after due' 
  end as results_status
from clinical_study cs
where
  -- include only trials that are required to submit results and who completed over a year ago. Trials are
  -- allowed 1 year from the primary completion date to submit results:
  -- 1. fits FDAAA 801 definition of an "applicable clinical trial"
  -- 2. initiated after 9/27/2007 
  --    or completed after 12/26/2017 even if initiated before 9/27/2007
  is_section_801 = 'Yes'
  AND (
    (
      start_date is not null 
      and start_date > '2007-09-27'
    )
    and (
      primary_completion_date is not null 
      and primary_completion_date > '2007-12-26'
      and primary_completion_date < '2015-06-01'
    )
    
  )
  -- investigational drugs which are seeking FDA approval are permitted to withold results until
  -- an approval decision has been made. According to the ClinicalTrials.gov xml schema, these studies
  -- will have a title of [Trial of device that is not approved or cleared by the U.S. FDA]. I'm excluding
  -- these studies from analysis because they are not required to post results at this time. 
  AND brief_title != '[Trial of device that is not approved or cleared by the U.S. FDA]'
  SQL
)

# 13,487 in total
regulated_studies.size

18047

In [16]:
overall_results_rates = regulated_studies.group_by(:results_status).count[1..1]
overall_results_rates.vectors = Daru::Index.new([:total_study_count])
overall_results_rates[:results_status] = overall_results_rates.index.to_a

overall_results_rates[:percent_of_total] = overall_results_rates.map(:row) do |r| 
  ((r[:total_study_count].to_f / regulated_studies.size.to_f) * 100.0).round(4)
end

overall_results_rates.plot type: :bar, x: :results_status, y: :total_study_count do |plot, diagram|
  plot.x_label "Results Status"
  plot.y_label "Total Studies"
end

#<Nyaplot::Frame:0x000000072e43a0 @properties={:panes=>[#<Nyaplot::Plot:0x000000072f47c8 @properties={:diagrams=>[#<Nyaplot::Diagram:0x000000072e5750 @properties={:type=>:bar, :options=>{:x=>:results_status, :y=>:total_study_count}, :data=>"5e0fd54e-636f-4f73-bba3-375d88275526"}, @xrange=["No results submitted", "Results submitted after due", "Results submitted on time"], @yrange=[0, 11084]>], :options=>{:x_label=>"Results Status", :y_label=>"Total Studies", :width=>700, :xrange=>["No results submitted", "Results submitted after due", "Results submitted on time"], :yrange=>[0, 11084]}}>], :data=>{"5e0fd54e-636f-4f73-bba3-375d88275526"=>#<Nyaplot::DataFrame:0x000000072e6858 @name="5e0fd54e-636f-4f73-bba3-375d88275526", @rows=[{:total_study_count=>11084, :results_status=>"No results submitted", :percent_of_total=>61.4174}, {:total_study_count=>4402, :results_status=>"Results submitted after due", :percent_of_total=>24.3919}, {:total_study_count=>2561, :results_status=>"Results submitted on time", :percent_of_total=>14.1907}]>}, :extension=>[]}>

In [15]:
overall_results_rates

#<Daru::DataFrame(3x3)>
            total_stud results_st percent_of
 No results      11084 No results    61.4174
 Results su       4402 Results su    24.3919
 Results su       2561 Results su    14.1907

11,084 (61.4%) of these studies have no results posted. 4,401 (24.39%) have submitted results but not until after the due date. Finally 2561 (14.2%) posted results within the 1 year period following completion of the study.

This seems low but one further consideration to be taken is that some studies are terminated or suspended before ever enrolling any participants so perhaps some truly have no data to report. What happens if we limit the above plot to only include studies with a status of 'Completed' - meaning that it the study concluded normally.

In [18]:
completed_regulated_studies = regulated_studies.filter(:row) { |r| r[:overall_status] == 'Completed' }
completed_regulated_studies.size
overall_results_rates = completed_regulated_studies.group_by(:results_status).count[1..1]
overall_results_rates.vectors = Daru::Index.new([:total_study_count])
overall_results_rates[:results_status] = overall_results_rates.index.to_a

overall_results_rates[:percent_of_total] = overall_results_rates.map(:row) do |r| 
  ((r[:total_study_count].to_f / completed_regulated_studies.size.to_f) * 100.0).round(4)
end

overall_results_rates.plot type: :bar, x: :results_status, y: :total_study_count do |plot, diagram|
  plot.x_label "Results Status"
  plot.y_label "Total Studies"
end

#<Nyaplot::Frame:0x000000086e0db8 @properties={:panes=>[#<Nyaplot::Plot:0x000000086f50d8 @properties={:diagrams=>[#<Nyaplot::Diagram:0x000000086e2028 @properties={:type=>:bar, :options=>{:x=>:results_status, :y=>:total_study_count}, :data=>"ebc93abb-60b9-49f5-9c7b-e54b7041afd1"}, @xrange=["No results submitted", "Results submitted after due", "Results submitted on time"], @yrange=[0, 6734]>], :options=>{:x_label=>"Results Status", :y_label=>"Total Studies", :width=>700, :xrange=>["No results submitted", "Results submitted after due", "Results submitted on time"], :yrange=>[0, 6734]}}>], :data=>{"ebc93abb-60b9-49f5-9c7b-e54b7041afd1"=>#<Nyaplot::DataFrame:0x000000086e3270 @name="ebc93abb-60b9-49f5-9c7b-e54b7041afd1", @rows=[{:total_study_count=>6734, :results_status=>"No results submitted", :percent_of_total=>54.544}, {:total_study_count=>3547, :results_status=>"Results submitted after due", :percent_of_total=>28.73}, {:total_study_count=>2065, :results_status=>"Results submitted on time", :percent_of_total=>16.7261}]>}, :extension=>[]}>

In [22]:
completed_regulated_studies.filter(:row) { |r| r[:results_status] == 'No results submitted' }.first(10)

#<Daru::DataFrame(10x8)>
            firstrecei is_section     nct_id overall_st primary_co results_du results_st start_date
          8        nil        Yes NCT0016619  Completed 2010-01-01 2011-01-01 No results 2008-01-01
          9        nil        Yes NCT0021143  Completed 2010-04-01 2011-04-01 No results 2008-04-01
         10        nil        Yes NCT0021422  Completed 2013-01-01 2014-01-01 No results 2007-12-01
         24        nil        Yes NCT0036337  Completed 2011-02-01 2012-02-01 No results 2008-01-01
         26        nil        Yes NCT0036978  Completed 2011-12-01 2012-12-01 No results 2008-01-01
         33        nil        Yes NCT0039022  Completed 2011-05-01 2012-05-01 No results 2008-02-01
         37        nil        Yes NCT0039757  Completed 2014-10-01 2015-10-01 No results 2013-05-01
         38        nil        Yes NCT0039820  Completed 2011-06-01 2012-06-01 No results 2008-02-01
         42        nil        Yes NCT0040476  Completed 2011-06-01 2012-06-01 No results 2007-10-01
         48        nil        Yes NCT0041861  Completed 2009-03-01 2010-03-01 No results 2008-01-01